In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import re

In [2]:

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def remove_url(text):
    return re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '(Sebuah tautan)', text)

def remove_identity(text):
    # Menghapus banyak awalan yang menunjukkan identitas, walaupun banyak bagian caption yang terpotong sebagai konsekuensinya
    return re.sub(r'\n(?i)(Biro|Bidang|Kabinet|Divisi|Departemen|Himpunan|#|--|__|Our social media|Find more|Stay tune|Tetap terhubung|Contact us|Line|Email|Wa :|Laman|Website|Twitter|Instagram|Facebook|Youtube|Ask fm|FB|IG|@|Tiktok|Medium|KMTK FT UGM|KMTNTF FT UGM|KMTG|KMTETI FT UGM|KMTM FT-UGM|Pengurus KMTG|IKAMMA|HMTG FT UGM|Wiswakharman|Himakom UGM|HIMAGIKA FKKMK|Computer, Love, Life|Surel|Sipil\? Solid.).*', '\n\n', text)

In [3]:
separator = "<|endoftext|>"

In [4]:
list_caption = []
for file in os.listdir("Bahan Scraping"):
    with open("Bahan Scraping/"+file,  encoding="utf8") as fp:
        soup = BeautifulSoup(fp, 'html.parser')
        list_caption.extend( [i.get('alt') for i in soup.find_all('img') 
                            if re.search(r'\[(.*?)\]', i.get('alt',""))]) 
list_caption = list(dict.fromkeys(list_caption))

In [5]:
df = pd.DataFrame(list_caption, columns=['raw_caption'])


#title is inside bracket
df['title'] = df['raw_caption'].str.extract(r'\[(.*?)\]', expand=False)

#Remove title from caption, remove leading newline, remove emoji
df['content'] = df['raw_caption'].apply(lambda x: deEmojify(x)).apply(lambda x: remove_url(x)).str.split(r'\[.*?\]').apply(lambda x: x[-1]).apply(lambda x: x.lstrip()).apply(lambda x: remove_identity(x)).apply(lambda x : re.sub(r'\n+', '\n',x))

df['processed'] = df.apply(lambda x: x['title']+" - "+x['content'], axis=1)


C:\Users\KARUNI~1\AppData\Local\Temp/ipykernel_7156/3390151365.py:15: DeprecationWarning: Flags not at the start of the expression '\\n(?i)(Biro|Bidang|K' (truncated)
  return re.sub(r'\n(?i)(Biro|Bidang|Kabinet|Divisi|Departemen|Himpunan|#|--|__|Our social media|Find more|Stay tune|Tetap terhubung|Contact us|Line|Email|Wa :|Laman|Website|Twitter|Instagram|Facebook|Youtube|Ask fm|FB|IG|@|Tiktok|Medium|KMTK FT UGM|KMTNTF FT UGM|KMTG|KMTETI FT UGM|KMTM FT-UGM|Pengurus KMTG|IKAMMA|HMTG FT UGM|Wiswakharman|Himakom UGM|HIMAGIKA FKKMK|Computer, Love, Life|Surel|Sipil\? Solid.).*', '\n\n', text)


In [6]:
print(df.sample(1)['processed'].values[0])

Kamis Puitis - Suramnya angkasa seakan turut berempati
Menani diri yang diselimuti sepi
Entah untuk yang ke berapa kali
Malam seolah penuh dengan ilusi
Tersenyum ia mempermainkan emosi
Kecamuk hati kian waktu kian menjadi
Merenung sejenak meratapi
Kini tiada rumah tuk menepi
Tiada topangan tuk berdiri
Juga tiada lagi hati yang mengisi
Sudah,
Kali ini,
Genap aku sendiri
Oleh: Tiara Dina



In [7]:
df['processed'].to_csv("list_caption.csv", index=False)